In [ ]:
# # !pip install -U langgraph langsmith langchain_anthropic 
# !pip install -qU langchain-openai
# !pip install openai
# !pip install faiss-cpu
# !pip install langchain_community
# !pip install pdfplumber
# !pip install langchain_huggingface
# !pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu124

# Setting

In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
from langchain_teddynote import logging
logging.langsmith('TailorLink_QA')

LangSmith 추적을 시작합니다.
[프로젝트명]
TailorLink_QA


In [3]:
import faiss
from langchain_community.vectorstores import FAISS

def load_faiss(embeddings):
    
    current_dir = os.getcwd()
    
    # 상대 경로를 기반으로 한 절대 경로 생성
    folder_path = os.path.join(current_dir, "pdf", "faiss_db", "faiss_db")
    index_name = os.path.join(current_dir, "pdf", "faiss_db", "faiss_index")
    
    loaded_db = FAISS.load_local(
        folder_path=folder_path,
        index_name=index_name,
        embeddings=embeddings,
        allow_dangerous_deserialization=True,
    )
    return loaded_db

In [4]:
from langchain_community.document_loaders import PDFPlumberLoader
from langchain_core.documents import Document

In [5]:
import pdfplumber
import json
def LoadPDF2(file_path: str):
    docs = []
    # PDFPlumber를 사용하여 파일을 로드합니다.
    with pdfplumber.open(file_path) as pdf:
        for page in pdf.pages:
            # 페이지 크기 확인
            width, height = page.width, page.height

            # 2개의 영역(열)으로 나누기
            left_bbox = (0, 0, width / 2, height)   # 왼쪽 열
            right_bbox = (width / 2, 0, width, height)  # 오른쪽 열

            # 왼쪽 열 텍스트 추출
            left_text = page.within_bbox(left_bbox).extract_text() or ""
            
            # 오른쪽 열 텍스트 추출
            right_text = page.within_bbox(right_bbox).extract_text() or ""

            # 텍스트 병합
            page_text = left_text + "\n" + right_text

            # 문서를 Langchain 형식으로 변환하여 추가
            if page_text.strip():
                docs.append(Document(
                    page_content=page_text,
                    metadata={
                        'source': file_path,
                        'file_path': file_path,
                        'page': page.page_number,
                        'total_pages': len(pdf.pages),
                        'CreationDate': pdf.metadata.get('CreationDate', None),
                        'ModDate': pdf.metadata.get('ModDate', None)
                    }
                ))

    return docs


In [6]:
docs = LoadPDF2('../data/genesis/genesis-g90-black-24-manual-kor.pdf')


In [7]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [8]:
def text_split(docs, chunk_size=100, chunk_overlap=30):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
    split_documents = text_splitter.split_documents(docs)
    print(f"분할된 청크의수 : {len(split_documents)}")
    return split_documents

In [9]:
docs = text_split(docs)

분할된 청크의수 : 8301


In [10]:
from langchain_core.prompts import PromptTemplate
# 프롬프트를 생성합니다.

prompt = PromptTemplate.from_template(
        """You are an assistant for question-answering tasks.
    Use only the following pieces of retrieved context to answer the question. 
    Do not use any outside knowledge or assumptions. 
    If the context does not provide enough information to answer, just say that you don't know. 
    Answer in Korean and format your answer.

    When answering, follow these detailed steps:
    1. **Summarize** the key points from the context relevant to the question in a concise manner.
    2. **Analyze** how the context connects to the question being asked.
    3. **Formulate** a clear and structured answer using the information in the context.
    4. If the context does not provide enough information, explicitly state: "제가 알 수 있는 정보가 부족합니다."
    5. Ensure your answer is well-organized and formatted for readability.

    #Context: 
    {context}

    #Question:
    {question}

    #Answer:"""
)


In [11]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini")

In [12]:
# 경고 무시
import os
import warnings
warnings.filterwarnings("ignore")
# ./cache/ 경로에 다운로드 받도록 설정
os.environ["HF_HOME"] = "./cache/"

In [13]:
from langchain_huggingface import HuggingFaceEmbeddings

def get_huggingface_embedding():
    # model_name = "BAAI/bge-m3"
    model_name = "intfloat/multilingual-e5-large-instruct"
    model_kwargs = {"device": "cuda"}
    encode_kwargs = {"normalize_embeddings": True}
    hf_embeddings = HuggingFaceEmbeddings(
        model_name=model_name, model_kwargs=model_kwargs, encode_kwargs=encode_kwargs
    )
    return hf_embeddings

hf_embeddings = get_huggingface_embedding()

In [15]:
# faiss = load_faiss(hf_embeddings)

In [16]:
from langchain_community.docstore.in_memory import InMemoryDocstore

dimension_size = len(hf_embeddings.embed_query("hello world"))

faiss = FAISS(
    embedding_function=hf_embeddings,
    index=faiss.IndexFlatL2(dimension_size),
    docstore=InMemoryDocstore(),
    index_to_docstore_id={},
    )
faiss.add_documents(
        docs,
    )

['67f6bdc0-0c60-42c8-a146-90cef1fa37a1',
 '1c9c2131-6e13-46d7-bf3f-8edd4e227ff0',
 '7c344c0d-dcf7-4087-a0c1-e9aa39be1025',
 '658bba10-79d0-45ee-8551-2170767459c6',
 '3df53e94-1852-47c4-8714-1af4435d59d3',
 'db9e1ab6-d469-44ff-8696-c7705af5d012',
 '8ac1c668-b74b-4964-8d79-2813c0c94886',
 '1cef41bc-fbbf-4910-8993-62073cfc9a9e',
 'ad81880d-78ba-407f-8b57-b777f504f9fb',
 '592ff919-d942-4d8d-90fe-952bab32767d',
 '89a3c363-751b-4a8c-819b-cf14e4d3def9',
 'a0002b6e-5197-4440-9cdf-f289c1e28ce4',
 '885e5092-6d16-4912-942f-ce3caf2ad8fa',
 '22eb8f9c-12aa-47aa-990a-077a22f9ba5e',
 'e788f5bb-7d8e-45d9-b8a2-83be85b904f2',
 '38d19a58-621c-491c-b9a7-ef7dd0b7aa4b',
 'f94686a7-2e06-4293-b215-2fb00529af7e',
 '15bfd00a-a9e2-46a2-aaf4-20901a26e0c9',
 'd0b1a9de-b10b-4bb4-876f-75d5c1300735',
 '8270def2-fae4-4b2c-9357-e4338244740b',
 '086ef787-7c3c-40b4-a7ae-f94479e3c467',
 'accface0-1564-4651-9e61-4e34acaf32b6',
 '88a60c1e-1697-4354-a7b1-a8a2767ba419',
 'c9f9aa00-f7ac-45f4-91bd-edb223706549',
 '1b276aca-01de-

In [17]:
from langchain.retrievers import EnsembleRetriever
from langchain_community.retrievers import BM25Retriever    

In [18]:
retriever = faiss.as_retriever(search_kwargs={"k": 10})

In [19]:
bm25 = BM25Retriever.from_documents(docs)


bm25_faiss_73 = EnsembleRetriever(
    retrievers=[bm25, retriever],  # 사용할 검색 모델의 리스트
    weights=[0.5, 0.5],  # 각 검색 모델의 결과에 적용할 가중치
    search_type="mmr",  # 검색 결과의 다양성을 증진시키는 MMR 방식을 사용
)


In [20]:
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import CrossEncoderReranker
from langchain_community.cross_encoders import HuggingFaceCrossEncoder

reranker = HuggingFaceCrossEncoder(model_name="BAAI/bge-reranker-v2-m3")
#상위 3개의 문서 선택
compressor = CrossEncoderReranker(model=reranker, top_n=10)

# 문서 압축 검색기 초기화
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor, base_retriever=bm25_faiss_73
)

In [21]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
chain = (
        {"context": compression_retriever, "question": RunnablePassthrough()}
        | prompt
        | llm
        | StrOutputParser()
    )

In [22]:
import pandas as pd


from llama_index.llms.openai import OpenAI
from autorag.data.legacy.qacreation import make_single_content_qa, generate_qa_llama_index

corpus_df = pd.read_parquet('../data/test_dataset/corpus_new_test.parquet', engine='pyarrow')


In [23]:
corpus_df['query'][0]

'고속도로 주행 보조 기능은 어떻게 작동하나요?'

In [24]:
corpus_df

,retrieval_gt,qid,query,generation_gt
0,[[551b1b32-74f4-47a5-93f4-525bd3813df9]],d16f89e4-2c63-429a-b3f4-42bccf7a568e,고속도로 주행 보조 기능은 어떻게 작동하나요?,[고속도로 주행 보조는 작동 가능한 도로 주행 시 주행 보조 버튼을 눌러 켜면 작동...
1,[[551b1b32-74f4-47a5-93f4-525bd3813df9]],3e36cd8d-e4b9-47aa-a0fb-1860b2ef3ff4,고속도로 주행 보조 작동 중 앞 차량이 정차하면 어떻게 되나요?,"[고속도로 주행 보조 작동 중 앞 차량이 정차하면 따라서 정차하며, 정차 후 약 3..."
2,[[dd8f42ca-2900-4e58-8628-a30a2ed6cf07]],da06362e-5640-40dd-893d-5ac8da33b580,제네시스 차량의 지능형 속도 제한 보조 기능은 어떤 상황에서 작동하나요?,[지능형 속도 제한 보조 기능은 도로의 제한속도가 70 km/h 이상인 경우에 작동...
3,[[dd8f42ca-2900-4e58-8628-a30a2ed6cf07]],a888eac0-5851-4ea7-91f6-45052c46dc2f,제네시스 차량에서 전방 카메라 렌즈에 이물질이 묻었을 때 어떻게 해야 하나요?,[전방 카메라 렌즈에 이물질이 묻으면 인식 성능이 저하되어 지능형 속도 제한 보조가...
4,[[b027a11d-1169-423e-a321-d1338e25cd46]],f8db5c64-c267-4558-bcb6-a631d57106bf,주차 거리 경고 기능이 작동하지 않을 때 어떤 점검을 해야 하나요?,"[주차 거리 경고 기능이 작동하지 않을 때는 초음파센서가 손상되었는지, 외부 물체에..."
5,[[b027a11d-1169-423e-a321-d1338e25cd46]],45891ecb-36fb-4c8f-9f62-633edbea56e4,주차 거리 경고 기능이 정상적으로 작동하지 않는 경우 어떤 상황이 있을까요?,[주차 거리 경고 기능이 정상적으로 작동하지 않는 경우는 초음파센서 표면에 눈이나 ...
6,[[79b3b76e-936f-43b8-8e17-3f517a8ece1e]],b50568de-3348-452d-be1b-d9a10243d7be,제네시스 차량의 에어백은 어떤 상황에서 작동하나요?,[제네시스 차량의 에어백은 정면에서 보통 이상 강도로 충돌하거나 측면에서 보통 이상...
7,[[79b3b76e-936f-43b8-8e17-3f517a8ece1e]],4881d2de-817d-4988-842e-072536b1dcd3,제네시스 차량에서 어린이를 안전하게 보호하기 위한 권장 사항은 무엇인가요?,"[제네시스 차량에서는 어린이를 뒷좌석에 설치된 어린이 보호 좌석에 앉히고, 동승석에..."
8,[[5bec5dde-6663-416c-8c75-5c594dcd91f6]],c74fdeae-9850-4593-8f05-c31f8a86e1e7,제네시스 차량의 후방 뷰 주차 가이드라인은 어떤 거리를 나타내나요?,"[후방 뷰 주차 가이드라인은 차량으로부터 0.5 m, 1 m, 2.3 m 거리를 나..."
9,[[5bec5dde-6663-416c-8c75-5c594dcd91f6]],2353a169-643e-4968-bc14-4da855246e71,서라운드 뷰 모니터의 자동 켜짐 기능을 설정하는 방법은 무엇인가요?,[서라운드 뷰 모니터의 자동 켜짐 기능을 설정하려면 시동 'ON' 상태에서 인포테인...


In [25]:
from evaluation.evaluation import * 
def evaluate_rag1(question, answer, chain, use_g_eval=False, use_gpu=False):
    res = chain.invoke(question)
    print(f"question: {question}")
    print(f"answer: {answer}")
    print(f"predict: {res}")
    
    return evaluate_rag(answer, res, use_g_eval, use_gpu)
    

In [26]:
a = evaluate_rag1(corpus_df['query'][1],corpus_df['generation_gt'][1][0], chain, True, True)
print(a)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

[12/05/24 11:55:59] INFO     [_client.py:1026] >> HTTP Request: POST                                ]8;id=165986;file://E:\TailorLink\SKN03-FINAL-6Team\TailorLink_LLM\doc_retrieval_qa\.venv\Lib\site-packages\httpx\_client.py\_client.py]8;;\:]8;id=617830;file://E:\TailorLink\SKN03-FINAL-6Team\TailorLink_LLM\doc_retrieval_qa\.venv\Lib\site-packages\httpx\_client.py#1026\1026]8;;\
                             https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"                          

question: 고속도로 주행 보조 작동 중 앞 차량이 정차하면 어떻게 되나요?
answer: 고속도로 주행 보조 작동 중 앞 차량이 정차하면 따라서 정차하며, 정차 후 약 30초 내에 앞 차량이 출발하면 자동으로 출발합니다. 30초가 지나면 클러스터에 안내문이 표시됩니다.
predict: 1. **요약**: 고속도로 주행 보조가 작동 중일 때, 앞 차량이 정차하면 해당 차량도 자동으로 정차하며, 이후 약 30초 후에 재출발합니다.

2. **분석**: 질문은 고속도로 주행 보조 작동 시 앞 차량이 정차했을 때의 차량의 동작을 묻고 있습니다. 제공된 정보는 이 상황에서 차량의 행동을 명확히 설명하고 있습니다.

3. **답변**: 고속도로 주행 보조 작동 중에 앞 차량이 정차하면, 차량은 자동으로 정차하며, 정차 후 약 30초 후에 재출발합니다.

4. **결론**: 질문에 대한 명확한 답변이 제공되었습니다.
calculating scores...
computing bert embedding.


  0%|          | 0/1 [00:00<?, ?it/s]

computing greedy matching.


...

[12/05/24 11:56:00] INFO     [SentenceTransformer.py:218] >> Load pretrained             ]8;id=256648;file://E:\TailorLink\SKN03-FINAL-6Team\TailorLink_LLM\doc_retrieval_qa\.venv\Lib\site-packages\sentence_transformers\SentenceTransformer.py\SentenceTransformer.py]8;;\:]8;id=788354;file://E:\TailorLink\SKN03-FINAL-6Team\TailorLink_LLM\doc_retrieval_qa\.venv\Lib\site-packages\sentence_transformers\SentenceTransformer.py#218\218]8;;\
                             SentenceTransformer:                                                                  
                             sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2                           

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

[12/05/24 11:56:06] INFO     [_client.py:1026] >> HTTP Request: POST                                ]8;id=819571;file://E:\TailorLink\SKN03-FINAL-6Team\TailorLink_LLM\doc_retrieval_qa\.venv\Lib\site-packages\httpx\_client.py\_client.py]8;;\:]8;id=852863;file://E:\TailorLink\SKN03-FINAL-6Team\TailorLink_LLM\doc_retrieval_qa\.venv\Lib\site-packages\httpx\_client.py#1026\1026]8;;\
                             https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"                          

{'METEOR': {'recall': 0.6, 'precision': 0.2112676056338028, 'f1_score': 0.5067567567567568}, 'ROUGE-1': {'recall': 0.6, 'precision': 0.2112676056338028, 'f1_score': 0.3125}, 'ROUGE-L': {'recall': 0.48, 'precision': 0.16901408450704225, 'f1_score': 0.25}, 'BERTScore': {'recall': 0.7794641256332397, 'precision': 0.7031147480010986, 'f1_score': 0.7393235564231873}, 'Precision': {'METEOR': 0.2112676056338028, 'ROUGE-1': 0.2112676056338028, 'ROUGE-L': 0.16901408450704225, 'BERTScore': 0.7031147480010986}, 'Recall': {'METEOR': 0.6, 'ROUGE-1': 0.6, 'ROUGE-L': 0.48, 'BERTScore': 0.7794641256332397}, 'F1 Score': {'METEOR': 0.5067567567567568, 'ROUGE-1': 0.3125, 'ROUGE-L': 0.25, 'BERTScore': 0.7393235564231873}, 'Cosine Similarity': {'Cosine Similarity': 0.83502215}, 'G-EVAL': '비율 점수: **85**\n\n이 유사성 점수는 두 텍스트 간의 의미와 정보의 일치성을 반영합니다. \n\n이유:\n1. **정보의 일치성**: 두 텍스트 모두 고속도로 주행 보조 시스템이 작동 중일 때 앞 차량이 정차하면 후속 차량도 정차하며, 이후 30초 경과 후에 재출발한다는 점을 공통적으로 설명하고 있습니다.\n2. **언어의 차이**: 원문은 비교적 간결하고 직접적인 설명을 제공하는 

# Langgraph

In [ ]:
from typing import Annotated

from typing_extensions import TypedDict

from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages


class State(TypedDict):
    # Messages have the type "list". The `add_messages` function
    # in the annotation defines how this state key should be updated
    # (in this case, it appends messages to the list, rather than overwriting them)
    messages: Annotated[list, add_messages]


graph_builder = StateGraph(State)

In [ ]:
from langchain_openai import ChatOpenAI

# llm = ChatOpenAI(model="gpt-4o-mini")

def searchFAQ(state: State):
    print(state)
    return {"messages": [chain.invoke(state["messages"][0].content)]}

def searchVectorDB(state: State):
    return {"messages": [chain.invoke(state["messages"])]}

graph_builder.add_node("searchFAQ", searchFAQ)
graph_builder.add_node("searchVectorDB", searchVectorDB)

In [ ]:
graph_builder.add_edge(START, "searchFAQ")
# graph_builder.add_edge("searchFAQ", END)
graph_builder.add_edge("searchFAQ", "searchVectorDB")
graph_builder.add_edge("searchVectorDB", END)
graph = graph_builder.compile()

In [ ]:
from IPython.display import Image, display

try:
    display(Image(graph.get_graph().draw_mermaid_png()))
except Exception:
    # This requires some extra dependencies and is optional
    pass

In [ ]:
graph.invoke({'messages': '브레이크는 뭐야?'})

In [ ]:
graph.invoke({'messages': ['브레이크는 뭐야?', '시동은 어떻게 켜?']})


In [ ]:


import pandas as pd
from llama_index.llms.openai import OpenAI
from autorag.data.legacy.qacreation import make_single_content_qa, generate_qa_llama_index

corpus_df = pd.read_parquet('../data/test_dataset/corpus_new_test.parquet', engine='pyarrow')


In [ ]:
from evaluation.evaluation import evaluate_rag 



In [ ]:
corpus_df['generation_gt']

In [ ]:

res = chain.invoke(corpus_df['query'][0])


In [ ]:

res

In [ ]:
evaluate_rag(corpus_df['generation_gt'][0][0], res, True, True)